# 类型

类型确定一组值以及特定于这些值的操作和方法。例如数字类型的值有加、减、乘、除操作，字符串类型的值有拼接操作。

类型可以用类型名称表示（如果有），或者从现有类型组成一个类型。

预先声明的类型：

    bool byte complex64 complex128 error float32 float64
    int int8 int16 int32 int64 rune string
    uint uint8 uint16 uint32 uint64 uintptr
    
组成的类型例如：

数组 `[23]int`, 结构 `struct { x int }`, 映射 `map[string]int`, 函数 `func(int, int) int` 等。

每个类型都有一个基础类型。上述预先声明的类型和组成的类型，其基础类型是它自身。

通过类型声明引入的类型，它的基础类型是在其类型声明中引用的类型的基础类型。

## 类型声明

类型声明将标识符（类型名称）绑定到类型。类型声明有两种形式：别名声明和类型定义。

别名声明：

In [2]:
type newint = int // newint 是 int 的别名，是同一个类型

type (
    aint = newint 
    bint = aint
) 
// aint，bint 和 newint 都是 int 的别名，基础类型都是 int

In [3]:
bint(1)

1

类型定义：

类型定义创建一个新的、不同的类型，称为 *定义类型（defined type）*，其基础类型和操作与给定类型相同，并将标识符绑定到该类型：

In [4]:
type myint int // myint 是一个新类型
type youint myint // youint 也是一个新类型
// myint, youint 和 int 基础类型都是 int

In [5]:
myint(1) + youint(1)

2

定义类型不会继承绑定到给定类型的任何方法，但接口类型或复合类型的元素的方法集保持不变：

In [7]:
type myint int

// 给 myint 创建一个新方法
func (m myint) Add() myint {
    return m + 1
}

myint(1).Add()

2

In [8]:
type youint myint

youint(1).Add() // 不会继承

ERROR: repl.go:3:1: invalid qualified type, expecting packagename.identifier, found: youint(1).Add <*ast.SelectorExpr>

In [9]:
type newstruct struct { myint } // myint 是复合类型 struct { myint } 的元素

a := newstruct{1}
a.myint.Add() // 元素的方法集保持不变

2

## 类型和值的属性

定义类型始终与任何其他类型不同。

如果两个类型的基础类型在结构上是等效的，则两个类型是相同的：

- 如果两个数组类型具有相同的元素类型和相同的数组长度，则它们是相同的。
- 如果两个切片类型具有相同的元素类型，则它们是相同的。
- 如果两个结构类型具有相同的字段序列，并且相应的字段具有相同的名称、相同的类型和相同的标记，则它们是相同的。来自不同包的非导出字段名称始终不同。
- 如果两个指针类型具有相同的基类型，则它们是相同的。
- 如果两个函数类型具有相同的参数和结果值数，则它们是相同的，相应的参数和结果类型是相同的，并且两个函数要么是可变参数的，要么两个函数都不是可变参数。参数和结果名称不需要匹配。
- 如果两个接口类型具有相同的方法集，并且具有相同的名称和相同的函数类型，则它们是相同的。来自不同包的未导出方法名称始终不同。方法的顺序无关紧要。
- 如果两个映射类型具有相同的键和元素类型，则它们是相同的。
- 如果两个通道类型具有相同的元素类型和相同的方向，则它们是相同的。

In [10]:
type (
        A0 = []string
        A1 = A0
        A2 = struct{ a, b int }
        A3 = int
        A4 = func(A3, float64) *A0
        A5 = func(x int, _ float64) *[]string
)

type (
        B0 A0
        B1 []string
        B2 struct{ a, b int }
        B3 struct{ a, c int }
        B4 func(int, float64) *B0
        B5 func(x int, y float64) *A1
)

type    C0 = B0

这些类型是相同的：

    A0, A1, and []string
    A2 and struct{ a, b int }
    A3 and int
    A4, func(int, float64) *[]string, and A5

    B0 and C0
    []int and []int
    struct{ a, b *T5 } and struct{ a, b *T5 }
    func(x int, y float64) *[]string, func(int, float64) (result *[]string), and A5

如果满足以下条件之一，则值 x 可赋给类型为 T 的变量：

- 当 x 的类型和 T 相同时。
- 当 x 的类型 V 和 T 有相同的 基本类型 且在 V 或 T 中至少有一个不是 定义类型 时。
- 当 T 为接口类型且 x 实现了 T 时。
- 当 x 为双向信道值、T 为信道类型、 x 的类型 V 和 T 的元素类型相同且在 V 或 T 中至少有一个不是 定义类型 时。
- 当 x 为预声明标识符 nil 且 T 为指针、函数、切片、映射、信道或接口类型时。
- 当 x 为无类型化的，可通过类型 T 的值来表示的常量时。

## 类型转换

常量值 x 在这些情况下可转换为类型 T：

- x 可表示为类型为 T 的值。
- x 为整数常量且 T 为 字符串类型。 

转换一个常量将产生一个类型化的常量作为结果：

    uint(iota)               // 类型为 uint 的 iota 值
    float32(2.718281828)     // 类型为 float32 的 2.718281828
    complex128(1)            // 类型为 complex128 的 1.0 + 0.0i
    string('x')              // 类型为 string 的 "x"
    string(0x266c)           // 类型为 string 的 "♬"
    MyString("foo" + "bar")  // 类型为 MyString 的 "foobar"
    string([]byte{'a'})      // 非常量：[]byte{'a'} 不为常量
    (*int)(nil)              // 非常量：nil 不为常量，*int 不为布尔、 数值或字符串类型
    int(1.2)                 // 非法：1.2 不能表示为 int
    string(65.0)             // 非法：65.0 不为整数常量
    
非常量值 x 在这些情况下可转换为类型 T：

- 当 x 可赋予 T 时。
- 当 x 的类型与 T 拥有相同的基础类型时。
- 当 x 的类型与指针类型 T 不是 定义类型，且它们的指针基类型拥有相同的基础类型时。
- 当 x 的类型与 T 同为整数或浮点数类型时。
- 当 x 的类型与 T 同为复数类型时。
- 当 x 为整数、字节切片或符文切片且 T 为字符串类型时。
- 当 x 为字符串且 T 为字节切片或符文切片时。
- 当 x 是一个切片，T 是指向数组的指针，并且切片和数组类型具有相同的元素类型。

对于非常量数值类型的类型转换，以下规则适用：

- 当在整数类型间转换时，若该值为无符号整数，其符号将扩展为隐式无限精度，反之为零扩展。 然后截断以符合该返回类型的大小。例如，若 v := uint16(0x10F0)，则 uint32(int8(v)) == 0xFFFFFFF0。类型转换总产生有效值，且无溢出指示。
- 当转换浮点数为整数时，小数部分将被丢弃（向零截断）。
- 当转换整数或浮点数为浮点类型，或转换复数类型为另一个复数类型时，其返回值将舍入至目标类型指定的精度。 例如，类型为 float32 的变量 x 的值可能使用超出 IEEE-754 标准32位数的额外精度来存储，但 float32(x) 表示将 x 的值舍入为32位精度的结果。 同样，x + 0.1 会使用超过32位的精度，但 float32(x + 0.1) 却不会。

字符串类型的转换：

- 将有符号或无符号整数值转换为字符串类型将产生一个包含 UTF-8 表示的该整数的字符串。有效 Unicode 码点范围之外的值将转换为 "\uFFFD"。

In [2]:
string('a')       // "a"
string(-1)        // "\ufffd" == "\xef\xbf\xbd "
string(0xf8)      // "\u00f8" == "ø" == "\xc3\xb8"
type MyString string
MyString(0x65e5)  // "\u65e5" == "日" == "\xe6\x97\xa5"

日

- 将字节切片转换为字符串类型将产生一个连续字节为该切片元素的字符串。若该切片值为 nil，则其结果为空字符串。

In [3]:
string([]byte{'h', 'e', 'l', 'l', '\xc3', '\xb8'})  // "hellø"

type MyBytes []byte
string(MyBytes{'h', 'e', 'l', 'l', '\xc3', '\xb8'})  // "hellø"

hellø

- 将符文切片转换为字符串类型将产生一个已转换为字符串的单个符文值的串联字符串。若该切片值为 nil，则其结果为空字符串。

In [4]:
string([]rune{0x767d, 0x9d6c, 0x7fd4})  // "\u767d\u9d6c\u7fd4" == "白鵬翔"

type MyRunes []rune
string(MyRunes{0x767d, 0x9d6c, 0x7fd4})  // "\u767d\u9d6c\u7fd4" == "白鵬翔"

白鵬翔

- 将字符串类型值转换为字节类型切片将产生一个连续元素为该字符串字节的切片。若该字符串为空，其结果为 `[]byte(nil)`。

In [8]:
[]byte("hellø")   // []byte{'h', 'e', 'l', 'l', '\xc3', '\xb8'}
MyBytes("hellø")  // []byte{'h', 'e', 'l', 'l', '\xc3', '\xb8'}

[104 101 108 108 195 184]

- 将字符串类型的值转换为符文类型切片将产生一个包含该字符串单个 Unicode 码点的切片。若该字符串为空，其结果为 `[]rune(nil)`。

In [9]:
[]rune(MyString("白鵬翔"))  // []rune{0x767d, 0x9d6c, 0x7fd4}
MyRunes("白鵬翔")           // []rune{0x767d, 0x9d6c, 0x7fd4}

[30333 40300 32724]